In [22]:
import numpy as np
import os
import tempfile
import keras
from keras import backend as K
from keras import layers
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
from keras.callbacks import History 
import matplotlib.pyplot as plt
import pickle



In [23]:
# UPLOAD DATA
# (each user should put datafiles in this directory on their computer)
datapath = "blast_tab_1hit.out"
file = open(datapath, 'r')

same_entries = []
diff_entries = []
max_length_in = 0
max_length_out = 0

for ln in file:
    toks = ln.split('\t')
    max_length_in = max(max_length_in,len(toks[2]))
    max_length_out = max(max_length_out,len(toks[3]))
    if toks[2] == toks[3]:
        same_entries.append([toks[2], toks[3]])
    else:
        diff_entries.append([toks[2], toks[3]])

file.close()
num_entries = len(same_entries) + len(diff_entries)



diff_entries_input = [entry[0] for entry in diff_entries]
diff_entries_output = [("\t" + entry[1] + "\n") for entry in diff_entries] #use '\t' as start character and '\n' as end character
#Visualize
diff_entries_output[1]
one_hot_input = {'A': 0, 'T': 1, 'C': 2, 'G': 3, '-': 4, '\t': 5, '\n': 6}
one_hot_output = {'A': 0, 'T': 1, 'C': 2, 'G': 3}

In [24]:
print(len(diff_entries_input))

58863


In [25]:
#Choose dataset size
numExamples = 50000
input_seqs = diff_entries_input[0:numExamples]
output_seqs = diff_entries_output[0:numExamples]

#ENCODE
encoder_input_data = np.zeros(
    (len(input_seqs), max_length_in, len(one_hot_input)),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_seqs), max_length_out, len(one_hot_input)),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_seqs), max_length_out, len(one_hot_input)),
    dtype='float32')

for i, (input_seqs, output_seqs) in enumerate(zip(input_seqs, output_seqs)):
    for t, char in enumerate(input_seqs):
        encoder_input_data[i, t, one_hot_input[char]] = 1.
    for t, char in enumerate(output_seqs):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, one_hot_input[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, one_hot_input[char]] = 1
            
print(encoder_input_data[1,1:100, :])

[[0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.

In [26]:
#DEFINE MODEL

# define number of tokens in the lexicon
num_encoder_tokens = len(one_hot_input)
num_decoder_tokens = len(one_hot_input)
latent_dim = 256

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
batch_size = 500
epochs = 100

learning_rates = [.001, .005, .01, .05]
for lr in learning_rates:
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

    # Run training
    history = History()

    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2, verbose = 1,
              callbacks = [history])
    model.save('s2s_lr' + str(lr) + '.h5')
    with open('history_lr' + str(lr), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 1873s 47ms/step - loss: 1.0054 - acc: 0.2667 - val_loss: 0.9578 - val_acc: 0.2746
Epoch 2/100
40000/40000 [==============================] - 2116s 53ms/step - loss: 0.9351 - acc: 0.2947 - val_loss: 0.9067 - val_acc: 0.3163
Epoch 3/100
40000/40000 [==============================] - 2216s 55ms/step - loss: 0.8561 - acc: 0.3662 - val_loss: 0.7747 - val_acc: 0.4301
Epoch 4/100
40000/40000 [==============================] - 2298s 57ms/step - loss: 0.7714 - acc: 0.4578 - val_loss: 0.7462 - val_acc: 0.4776
Epoch 5/100
40000/40000 [==============================] - 2355s 59ms/step - loss: 0.6301 - acc: 0.5525 - val_loss: 0.5328 - val_acc: 0.6168
Epoch 6/100
40000/40000 [==============================] - 2394s 60ms/step - loss: 0.8019 - acc: 0.4156 - val_loss: 0.9256 - val_acc: 0.3004
Epoch 7/100
40000/40000 [==============================] - 2434s 61ms/step - loss: 0.9048 - acc: 0.3160 

In [ ]:
#pickle.load(open( "history_lr0.001", "rb" ) )

In [ ]:
for lr in learning_rates:
    history = pickle.load(open()'history_lr' + str(lr) + ".001", "rb")
    plt.plot(range(0, epochs), history["loss"])
    plt.plot(range(0, epochs), history["val_loss"])